In [1]:
!pip install -q pyngrok

!pip install -q streamlit

!pip install -q streamlit_ace

     |████████████████████████████████| 745 kB 5.3 MB/s 
     |████████████████████████████████| 8.0 MB 5.2 MB/s 
     |████████████████████████████████| 75 kB 4.0 MB/s 
     |████████████████████████████████| 111 kB 60.5 MB/s 
     |████████████████████████████████| 4.3 MB 55.9 MB/s 
     |████████████████████████████████| 180 kB 61.7 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 124 kB 60.5 MB/s 
     |████████████████████████████████| 787 kB 50.6 MB/s 
     |████████████████████████████████| 370 kB 45.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.20 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.4.1 which is incompatible.
google-colab 1.0.0 requires ipython

In [2]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split

data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP/IMDB Dataset.csv')
data['review'] = data['review'].str.lower()
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", 
             "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during",
             "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", 
             "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into",
             "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or",
             "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", 
             "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's",
             "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up",
             "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's",
             "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've",
             "your", "yours", "yourself", "yourselves" ]

def remove_stopwords(data):
  data['review without stopwords'] = data['review'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
  return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result

data_without_stopwords = remove_stopwords(data)
data_without_stopwords['clean_review']= data_without_stopwords['review without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_review'] = data_without_stopwords['clean_review'].str.replace('[{}]'.format(string.punctuation), ' ')

reviews = data_without_stopwords['clean_review']

reviews_list = []
for i in range(len(reviews)):
  reviews_list.append(reviews[i])

sentiment = data_without_stopwords['sentiment']
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, sentiment)))

X_train, X_test,Y_train, Y_test = train_test_split(reviews_list, y, test_size=0.2, random_state = 45)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

@st.cache(allow_output_mutation=True)
def load_model():
  model=tf.keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/NLP/imdb_lstm.h5")
  return model
with st.spinner('Model is being loaded..'):
  model=load_model()

st.write("""
         # Sentiment Analysis With LSTM
         """
         )

# file = st.file_uploader("Please upload an brain scan file", type=["jpg", "png"])
# import cv2
# from PIL import Image, ImageOps
# import numpy as np
text = st.text_input("Please Enter Your Statement...")
st.set_option('deprecation.showfileUploaderEncoding', False)
def import_and_predict(sentence, model):
        X_test_indices = tokenizer.texts_to_sequences([sentence])
        print(X_test_indices)
        X_test_indices = tf.keras.preprocessing.sequence.pad_sequences(X_test_indices, maxlen=150, padding='post')
        print(X_test_indices)
        predict = model.predict(X_test_indices)
        if predict > 0.5:
          probabilty = predict
          prediction = "Positive"
        else: 
          probabilty = predict
          prediction = "Negative"
          
        # size = (180,180)    
        # image = ImageOps.fit(image_data, size, Image.ANTIALIAS)
        # image = np.asarray(image)
        # img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # #img_resize = (cv2.resize(img, dsize=(75, 75),    interpolation=cv2.INTER_CUBIC))/255.
        
        # img_reshape = img[np.newaxis,...]
    
        # prediction = model.predict(img_reshape)
        
        return probabilty, prediction
if text is None:
    st.text("Please Enter an Statement ")
else:
    # image = Image.open(file)
    # st.image(image, use_column_width=True)
    probabilty, prediction = import_and_predict(text, model)
    # score = tf.nn.softmax(predictions[0])
    # st.write(probabilty)
    # st.write(prediction)
    string = f"This Statement most likely is {prediction} with probabilty of {probabilty}..."
    st.success(string)
    # print(
    # "This image most likely belongs to {} with a {:.2f} percent confidence."
    # .format(class_names[np.argmax(score)], 100 * np.max(score)))


Writing app.py


In [3]:
!streamlit run app.py &>/dev/null&

In [4]:
from pyngrok import ngrok
 
public_url = ngrok.connect('8501')
public_url

<NgrokTunnel: "http://cff0-34-122-82-153.ngrok.io" -> "http://localhost:8501">